In [1]:
#imports
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from tensorflow.keras.layers import Activation, concatenate, Dense, Dropout, Embedding, Input, Reshape, Flatten
from tensorflow.keras.models import Model
from tensorflow.keras.utils import plot_model
from tensorflow.keras.preprocessing.text import Tokenizer
import os

os.chdir('C:/Users/Sean/Documents/Projects/hockey_proj')
pd.options.display.max_columns = None


In [2]:
#read in data from csv
all_df = pd.read_csv('all_events23.csv')


C:\Users\Sean\AppData\Local\Temp\ipykernel_13068\109595414.py:2: DtypeWarning: Columns (51,58) have mixed types. Specify dtype option on import or set low_memory=False.
  all_df = pd.read_csv('all_events23.csv')


In [3]:
#check out data
all_df.shape
all_df.event_type.unique()

array(['GAME_SCHEDULED', 'FACEOFF', 'HIT', 'SHOT', 'STOP', 'MISSED_SHOT',
       'GIVEAWAY', 'BLOCKED_SHOT', 'GOAL', 'TAKEAWAY', 'PENALTY',
       'PERIOD_END', 'GAME_END', 'CHALLENGE', 'SHOOTOUT_COMPLETE',
       'UNKNOWN', 'FAILED_SHOT_ATTEMPT', 'EARLY_INT_START',
       'EARLY_INT_END'], dtype=object)

In [5]:
#we want to keep shots on net and goals
shots_df = all_df[all_df.event_type.isin(['SHOT', 'GOAL'])]
shots_df.head()


,Unnamed: 0,X,xg,event_id,event_type,event,secondary_type,event_team,event_team_type,description,period,period_seconds,period_seconds_remaining,game_seconds,game_seconds_remaining,home_score,away_score,event_player_1_name,event_player_1_type,event_player_2_name,event_player_2_type,event_player_3_name,event_player_3_type,event_goalie_name,strength_state,strength_code,strength,game_winning_goal,empty_net,penalty_severity,penalty_minutes,event_idx,num_on,players_on,num_off,players_off,extra_attacker,x,y,x_fixed,y_fixed,shot_distance,shot_angle,home_skaters,away_skaters,home_on_1,home_on_2,home_on_3,home_on_4,home_on_5,home_on_6,home_on_7,away_on_1,away_on_2,away_on_3,away_on_4,away_on_5,away_on_6,away_on_7,home_goalie,away_goalie,game_id,event_player_1_id,event_player_1_link,event_player_1_season_total,event_player_2_id,event_player_2_link,event_player_2_season_total,event_player_3_id,event_player_3_link,event_player_3_season_total,event_player_4_id,event_player_4_name,event_player_4_link,event_player_4_type,period_type,ordinal_num,period_time,period_time_remaining,date_time,event_team_id,event_team_link,event_team_abbr,home_final,away_final,event_goalie_id,event_goalie_link,event_goalie_type,ids_on,ids_off,season,season_type,game_date,game_start,game_end,game_length,game_state,detailed_state,venue_name,venue_link,home_name,home_abbreviation,home_division_name,home_division_name_short,home_conference_name,home_id,away_name,away_abbreviation,away_division_name,away_division_name_short,away_conference_name,away_id,venue_id
4,5,5,0.025524,2.022020e+13,SHOT,Shot,Wrist Shot,San Jose Sharks,away,Timo Meier Wrist Shot saved by Juuse Saros,1,23,1177,23,3577,0,0,Timo.Meier,Shooter,Juuse.Saros,Goalie,NaN,NaN,Juuse.Saros,5v5,EV,Even,NaN,NaN,NaN,NaN,6,NaN,NaN,NaN,NaN,False,44.0,8.0,-44.0,-8.0,45.7,10.1,5,5,Filip.Forsberg,Mikael.Granlund,Matt.Duchene,Roman.Josi,Alexandre.Carrier,NaN,NaN,Scott.Harrington,Timo.Meier,Erik.Karlsson,Luke.Kunin,Tomas.Hertl,NaN,NaN,Juuse.Saros,James.Reimer,2022020001,8478414.0,/api/v1/people/8478414,NaN,8477424.0,/api/v1/people/8477424,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,REGULAR,1st,00:23,19:37,2022-10-07T18:06:53Z,28.0,/api/v1/teams/28,SJS,4.0,1.0,8477424.0,/api/v1/people/8477424,Goalie,NaN,NaN,20222023,R,2022-10-07,2022-10-07 14:00:00,2022-10-07 16:49:33,2H 49M 33S,Final,Final,O2 Czech Republic,/api/v1/venues/null,Nashville Predators,NSH,Central,CEN,Western,18,San Jose Sharks,SJS,Pacific,PAC,Western,28,NaN
11,12,12,0.063270,2.022020e+13,SHOT,Shot,Slap Shot,Nashville Predators,home,Mattias Ekholm Slap Shot saved by James Reimer,1,59,1141,59,3541,0,0,Mattias.Ekholm,Shooter,James.Reimer,Goalie,NaN,NaN,James.Reimer,5v5,EV,Even,NaN,NaN,NaN,NaN,15,NaN,NaN,NaN,NaN,False,-33.0,8.0,33.0,-8.0,56.6,8.1,5,5,Nino.Niederreiter,Ryan.McDonagh,Kiefer.Sherwood,Mattias.Ekholm,Ryan.Johansen,NaN,NaN,Logan.Couture,Mario.Ferraro,Kevin.Labanc,Marc.Edouard.Vlasic,Oskar.Lindblom,NaN,NaN,Juuse.Saros,James.Reimer,2022020001,8475218.0,/api/v1/people/8475218,NaN,8473503.0,/api/v1/people/8473503,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,REGULAR,1st,00:59,19:01,2022-10-07T18:07:29Z,18.0,/api/v1/teams/18,NSH,4.0,1.0,8473503.0,/api/v1/people/8473503,Goalie,NaN,NaN,20222023,R,2022-10-07,2022-10-07 14:00:00,2022-10-07 16:49:33,2H 49M 33S,Final,Final,O2 Czech Republic,/api/v1/venues/null,Nashville Predators,NSH,Central,CEN,Western,18,San Jose Sharks,SJS,Pacific,PAC,Western,28,NaN
12,13,13,0.451566,2.022020e+13,GOAL,Goal,Wrist Shot,Nashville Predators,home,"Kiefer Sherwood (1) Wrist Shot, assists: Matti...",1,61,1139,61,3539,1,0,Kiefer.Sherwood,Scorer,Mattias.Ekholm,Assist,Ryan.McDonagh,Assist,James.Reimer,5v5,EV,Even,False,False,NaN,NaN,16,NaN,NaN,NaN,NaN,False,-74.0,-5.0,74.0,5.0,15.8,18.4,5,5,Nino.Niederreiter,Ryan.McDonagh,Kiefer.Sherwood,Mattias.Ekholm,Ryan.Johansen,NaN,NaN,Logan.Couture,Mario.Ferraro,Kevin.Labanc,Marc.Edouard.Vlasic,Oskar.Lindblom,NaN,NaN,Juuse.Saros,James.Reimer,2022020001,8480748.0,/api/v1/people/8480748,1.0,8475218.0,/api/v1/people/8475218

In [6]:
#keep only useful columns 
shots_df = shots_df[['event_type', 'secondary_type', 'event_player_1_name', 'strength_code', 'x_fixed', 'y_fixed', 'shot_distance', 'shot_angle', 'xg']]
shots_df.head()

,event_type,secondary_type,event_player_1_name,strength_code,x_fixed,y_fixed,shot_distance,shot_angle,xg
4,SHOT,Wrist Shot,Timo.Meier,EV,-44.0,-8.0,45.7,10.1,0.025524
11,SHOT,Slap Shot,Mattias.Ekholm,EV,33.0,-8.0,56.6,8.1,0.063270
12,GOAL,Wrist Shot,Kiefer.Sherwood,EV,74.0,5.0,15.8,18.4,0.451566
14,SHOT,Wrist Shot,Colton.Sissons,EV,81.0,-15.0,17.0,61.9,0.054611
22,SHOT,Tip-In,Steven.Lorentz,EV,-72.0,-2.0,17.1,6.7,0.151387


In [7]:
#rename column and generate unique player_ids
shots_df = shots_df.rename(columns={'event_player_1_name': 'name'})
shots_df['pid'] = pd.factorize(shots_df['name'])[0]

In [8]:
#get key mapping pids to names
#get name index pairs from shots_df
unique_pairs = shots_df.drop_duplicates(subset=['name', 'pid'])
player_pid_mapping = unique_pairs.groupby('name')['pid'].unique().reset_index()
player_pid_mapping = pd.DataFrame(player_pid_mapping)
#convert pid to int
player_pid_mapping['pid'] = player_pid_mapping['pid'].astype(int)

In [9]:
#get dummies for shot type and strength_code
shots_df = pd.get_dummies(shots_df, columns = ['secondary_type', 'strength_code'], dtype=int)

In [10]:
#create goal_variable
shots_df['goal'] = np.where(shots_df['event_type'] == 'GOAL', 1, 0)

In [11]:
shots_df.head()

,event_type,name,x_fixed,y_fixed,shot_distance,shot_angle,pid,secondary_type_Backhand,secondary_type_Batted,secondary_type_Between Legs,secondary_type_Cradle,secondary_type_Deflected,secondary_type_Penalty Shot,secondary_type_Poke,secondary_type_Slap Shot,secondary_type_Snap Shot,secondary_type_Tip-In,secondary_type_Wrap-around,secondary_type_Wrist Shot,strength_code_EV,strength_code_PP,strength_code_SH,goal
4,SHOT,Timo.Meier,-44.0,-8.0,45.7,10.1,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0
11,SHOT,Mattias.Ekholm,33.0,-8.0,56.6,8.1,1,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0
12,GOAL,Kiefer.Sherwood,74.0,5.0,15.8,18.4,2,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,1
14,SHOT,Colton.Sissons,81.0,-15.0,17.0,61.9,3,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0
22,SHOT,Steven.Lorentz,-72.0,-2.0,17.1,6.7,4,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0


In [11]:
#scale numeric data to 0/1
scaler = MinMaxScaler()
num_cols = ['x_fixed', 'y_fixed', 'shot_distance', 'shot_angle', 'xg']
shots_df[num_cols] = scaler.fit_transform(shots_df[num_cols])

In [12]:
#convert player ids to a string 
shots_df.pid = shots_df.pid.astype('str')


In [13]:
#set up and run tokenizer
shooter_tokenizer = Tokenizer()

shooter_tokenizer.fit_on_texts(shots_df.pid)

In [14]:
#convert to sequences to feed neural net - all sequences are same length, no padding
shooters = shooter_tokenizer.texts_to_sequences(shots_df.pid)
shooters = [x[0] for x in shooters]
shooters = np.array(shooters)

In [15]:
#set target for fitting embeddings - goals
goal = np.array(shots_df.goal)
shot_info = shots_df.drop(columns=['pid', 'goal', 'name', 'event_type'], axis=1, inplace=False)
SHOT_COLS = shot_info.shape[1]
NUM_SHOOTERS = len(np.unique(shooters)) + 2
VEC_SIZE = 50

In [16]:
# the input layers
shooter_input = Input(shape=(1, ), name="shooter_input")
shot_input = Input(shape=(SHOT_COLS, ), name="shot_input")

# shooter layers
s1 = Embedding(NUM_SHOOTERS, VEC_SIZE, input_length=1)(shooter_input)
s2 = Flatten()(s1)
s3 = Dense(1, activation="sigmoid")(s2)

# shot layers
s4 = Dense(16, activation='relu')(shot_input)
s5 = Dense(8, activation='relu')(s4)

# concatenate both paths
concatenated = concatenate([s3, s5])

# output layer
output = Dense(1, activation='sigmoid')(concatenated)

# create model
model = Model(inputs=[shooter_input, shot_input], outputs=output)

In [18]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 shooter_input (InputLayer)  [(None, 1)]                  0         []                            
                                                                                                  
 embedding (Embedding)       (None, 1, 50)                43850     ['shooter_input[0][0]']       
                                                                                                  
 shot_input (InputLayer)     [(None, 19)]                 0         []                            
                                                                                                  
 flatten (Flatten)           (None, 50)                   0         ['embedding[0][0]']           
                                                                                              

In [17]:
#compile model
model.compile(optimizer="adam", loss = "binary_crossentropy", metrics = ['accuracy'])

In [18]:
X = [shooters, shot_info]
history = model.fit(X, goal, epochs=10)

Epoch 1/10


2718/2718 [==============================] - 5s 1ms/step - loss: 0.3452 - accuracy: 0.8974
Epoch 2/10
2718/2718 [==============================] - 3s 1ms/step - loss: 0.3248 - accuracy: 0.8975
Epoch 3/10
2718/2718 [==============================] - 4s 1ms/step - loss: 0.3227 - accuracy: 0.8975
Epoch 4/10
2718/2718 [==============================] - 3s 1ms/step - loss: 0.3215 - accuracy: 0.8975
Epoch 5/10
2718/2718 [==============================] - 3s 1ms/step - loss: 0.3208 - accuracy: 0.8975
Epoch 6/10
2718/2718 [==============================] - 4s 1ms/step - loss: 0.3204 - accuracy: 0.8975
Epoch 7/10
2718/2718 [==============================] - 3s 1ms/step - loss: 0.3200 - accuracy: 0.8975
Epoch 8/10
2718/2718 [==============================] - 3s 1ms/step - loss: 0.3198 - accuracy: 0.8975
Epoch 9/10
2718/2718 [==============================] - 3s 1ms/step - loss: 0.3196 - accuracy: 0.8975
Epoch 10/10
2718/2718 [==============================] - 3s 1ms/step - loss: 0.3

In [19]:
#get embeddings
embedding_layer = model.get_layer('embedding')
shooter_embeddings = embedding_layer.get_weights()[0]

In [20]:
print(shooter_embeddings)

[[-0.00854257  0.0182592   0.0252018  ...  0.04123202 -0.02235882
  -0.01431326]
 [ 0.14586087  0.06094237 -0.1623175  ... -0.06244105  0.12429902
  -0.08871685]
 [ 0.02716214 -0.07089347  0.02772129 ...  0.03379526 -0.01618552
  -0.01087855]
 ...
 [-0.17264412 -0.32900408  0.2465134  ...  0.22495952 -0.2881526
   0.1580145 ]
 [-0.20604101 -0.25387785  0.22761697 ...  0.24016187 -0.2481465
   0.21253054]
 [ 0.02811373 -0.03967841 -0.03842429 ... -0.02632177 -0.0174965
   0.01524561]]


In [21]:
# make the embed vectors a pandas dataframe
shooter_embeddings = pd.DataFrame(shooter_embeddings)
shooter_embeddings.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49
0,-0.008543,0.018259,0.025202,-0.015643,-0.017361,0.029388,-0.039974,-0.041736,0.048719,0.045873,-0.048108,0.003002,0.030425,0.045921,0.019378,0.028281,-0.028981,0.049207,-0.039698,-0.005741,0.001991,-0.000931,-0.011814,-0.032048,-0.041403,-0.036653,-0.014265,-0.005239,0.022444,0.043765,0.035352,0.033642,0.032344,-0.035033,-0.013289,-0.015758,0.036928,-0.030885,0.037390,-0.048515,-0.017312,0.044788,-0.025793,0.029436,0.015569,0.001244,0.024595,0.041232,-0.022359,-0.014313
1,0.145861,0.060942,-0.162317,0.116521,0.102375,-0.122120,-0.082303,0.034962,0.086558,0.091245,-0.120297,-0.050057,-0.183504,-0.117833,0.034199,-0.056183,-0.098333,-0.024634,0.049995,-0.046039,0.040207,-0.093161,0.094091,0.061611,0.094433,-0.086497,0.039598,0.105978,0.048808,0.049360,0.097101,0.082115,0.077424,0.058181,-0.090922,0.101726,0.044492,0.088789,-0.081331,0.098527,0.129745,-0.108056,-0.097394,0.068444,0.054562,-0.167739,0.107999,-0.062441,0.124299,-0.088717
2,0.027162,-0.070893,0.027721,-0.050814,-0.041862,0.059687,0.072320,-0.088384,-0.044163,0.030299,0.056952,0.038949,-0.020898,-0.004208,-0.095019,0.050847,0.082287,0.042993,-0.008876,0.026541,-0.098129,0.063506,-0.070664,-0.063470,-0.063600,0.086194,-0.033844,0.080197,-0.111511,0.017512,-0.080008,0.032316,-0.037722,-0.038124,0.068845,-0.036078,-0.051056,-0.108139,0.025899,-0.096434,-0.027515,0.101593,0.067431,-0.038790,-0.043204,-0.045183,-0.027483,0.033795,-0.016186,-0.010879
3,0.268365,0.229462,-0.212464,0.253238,0.279540,-0.258065,-0.279415,0.258875,0.240509,0.242060,-0.226719,-0.257509,-0.257206,-0.234686,0.222459,-0.253505,-0.280366,-0.296878,0.246198,-0.239777,0.244030,-0.261487,0.280880,0.271278,0.247669,-0.307330,0.250398,0.308837,0.305877,0.224752,0.245398,0.253656,0.250657,0.267921,-0.252904,0.281080,0.281106,0.310864,-0.289701,0.268175,0.224581,-0.288116,-0.237987,0.247059,0.241400,-0.247237,0.221769,-0.262184,0.267216,-0.166358
4,-0.088892,0.049287,-0.008095,0.050189,0.038796,-0.001803,0.030166,-0.004872,0.031296,0.028977,-0.007679,-0.056806,0.015333,0.030285,-0.039920,-0.022837,-0.042695,0.022855,-0.024501,0.045643,0.023810,-0.050221,0.016176,-0.030571,0.053614,-0.022395,-0.001223,0.045490,0.047564,0.011430,-0.025801,-0.011130,-0.030625,0.036144,0.036815,0.048444,0.015780,0.007020,-0.004680,0.033442,0.028940,-0.018325,0.054501,-0.001842,-0.045956,0.016427,-0.019842,-0.017858,0.002855,-0.001432


In [22]:
# a list of true shooter ids
shooter_id = {k:v for k, v in shooter_tokenizer.index_word.items()}
shooter_df = pd.DataFrame.from_dict(shooter_id, orient='index', columns=["playerid"])

,e1,e2,e3
1,-0.126772,-0.103765,-0.207424
2,-0.013892,0.023600,0.007576
3,-0.059690,-0.032343,-0.107803


In [23]:
# name the columns
shooter_embeddings.columns = ["e" + str(i + 1) for i in range(shooter_embeddings.shape[1])]

# align the data by index
shooter_embeddings = pd.merge(shooter_embeddings, shooter_df, how='inner', left_index=True, right_index=True)

In [27]:
# clean up the index so its the player
shooter_embeddings.index = shooter_embeddings.playerid

# the first few
shooter_embeddings.iloc[:3, :3]

AttributeError: 'DataFrame' object has no attribute 'playerid'

In [25]:
#rename column
shooter_embeddings = shooter_embeddings.rename(columns = {'playerid':'pid'})

In [28]:
shooter_embeddings.head()

,e1,e2,e3,e4,e5,e6,e7,e8,e9,e10,e11,e12,e13,e14,e15,e16,e17,e18,e19,e20,e21,e22,e23,e24,e25,e26,e27,e28,e29,e30,e31,e32,e33,e34,e35,e36,e37,e38,e39,e40,e41,e42,e43,e44,e45,e46,e47,e48,e49,e50,pid
playerid,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
96,0.145861,0.060942,-0.162317,0.116521,0.102375,-0.122120,-0.082303,0.034962,0.086558,0.091245,-0.120297,-0.050057,-0.183504,-0.117833,0.034199,-0.056183,-0.098333,-0.024634,0.049995,-0.046039,0.040207,-0.093161,0.094091,0.061611,0.094433,-0.086497,0.039598,0.105978,0.048808,0.049360,0.097101,0.082115,0.077424,0.058181,-0.090922,0.101726,0.044492,0.088789,-0.081331,0.098527,0.129745,-0.108056,-0.097394,0.068444,0.054562,-0.167739,0.107999,-0.062441,0.124299,-0.088717,96
179,0.027162,-0.070893,0.027721,-0.050814,-0.041862,0.059687,0.072320,-0.088384,-0.044163,0.030299,0.056952,0.038949,-0.020898,-0.004208,-0.095019,0.050847,0.082287,0.042993,-0.008876,0.026541,-0.098129,0.063506,-0.070664,-0.063470,-0.063600,0.086194,-0.033844,0.080197,-0.111511,0.017512,-0.080008,0.032316,-0.037722,-0.038124,0.068845,-0.036078,-0.051056,-0.108139,0.025899,-0.096434,-0.027515,0.101593,0.067431,-0.038790,-0.043204,-0.045183,-0.027483,0.033795,-0.016186,-0.010879,179
239,0.268365,0.229462,-0.212464,0.253238,0.279540,-0.258065,-0.279415,0.258875,0.240509,0.242060,-0.226719,-0.257509,-0.257206,-0.234686,0.222459,-0.253505,-0.280366,-0.296878,0.246198,-0.239777,0.244030,-0.261487,0.280880,0.271278,0.247669,-0.307330,0.250398,0.308837,0.305877,0.224752,0.245398,0.253656,0.250657,0.267921,-0.252904,0.281080,0.281106,0.310864,-0.289701,0.268175,0.224581,-0.288116,-0.237987,0.247059,0.241400,-0.247237,0.221769,-0.262184,0.267216,-0.166358,239
337,-0.088892,0.049287,-0.008095,0.050189,0.038796,-0.001803,0.030166,-0.004872,0.031296,0.028977,-0.007679,-0.056806,0.015333,0.030285,-0.039920,-0.022837,-0.042695,0.022855,-0.024501,0.045643,0.023810,-0.050221,0.016176,-0.030571,0.053614,-0.022395,-0.001223,0.045490,0.047564,0.011430,-0.025801,-0.011130,-0.030625,0.036144,0.036815,0.048444,0.015780,0.007020,-0.004680,0.033442,0.028940,-0.018325,0.054501,-0.001842,-0.045956,0.016427,-0.019842,-0.017858,0.002855,-0.001432,337
302,-0.042943,-0.030247,-0.013283,0.037826,-0.015664,0.019334,0.027897,0.025692,-0.014122,0.058668,-0.016180,-0.032696,-0.062556,-0.050210,-0.015610,-0.006681,0.003106,0.137039,0.006128,-0.020474,0.014034,0.043212,0.049592,0.022960,-0.030112,-0.043782,0.010503,0.043819,-0.006916,0.056317,0.002551,-0.035403,-0.004829,-0.028073,-0.006933,-0.022408,0.048272,0.006185,-0.050122,-0.018341,0.030431,0.022734,-0.007365,-0.018434,0.062929,0.016293,0.058953,0.029134,-0.011722,-0.083194,302


In [29]:
#export to 3d space
from sklearn.manifold import TSNE
shooter_tsne = TSNE(n_components=3).fit_transform(shooter_embeddings.iloc[:, :50])

In [30]:
shooter_embeddings.reset_index(drop=True, inplace=True)

,e1,e2,e3,e4,e5,e6,e7,e8,e9,e10,e11,e12,e13,e14,e15,e16,e17,e18,e19,e20,e21,e22,e23,e24,e25,e26,e27,e28,e29,e30,e31,e32,e33,e34,e35,e36,e37,e38,e39,e40,e41,e42,e43,e44,e45,e46,e47,e48,e49,e50,playerid
playerid,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
96,0.152841,-0.235352,-0.208972,-0.229480,0.152205,-0.138292,-0.160937,0.168056,-0.141138,0.221545,0.202763,0.177171,-0.161440,-0.208070,0.145056,-0.243878,0.216302,-0.188928,-0.243610,0.160543,0.156964,-0.151005,0.167247,0.202051,0.210489,0.135924,-0.172658,-0.191543,-0.185757,-0.179759,-0.198636,0.130101,0.181310,-0.107068,-0.221785,0.133545,0.231786,-0.226405,-0.148754,0.163326,-0.136656,-0.169800,-0.210699,0.169203,-0.083975,-0.150257,-0.158707,0.224356,0.198516,-0.221775,96
239,0.177515,-0.198818,-0.192883,-0.189295,0.221831,-0.202387,-0.143346,0.142166,-0.218605,0.222986,0.154694,0.165252,-0.175187,-0.179465,0.230013,-0.202168,0.154499,-0.219805,-0.212487,0.179024,0.193361,-0.130645,0.194108,0.192925,0.162755,0.216818,-0.185495,-0.225264,-0.200317,-0.204387,-0.240080,0.187986,0.230157,-0.113080,-0.153287,0.162664,0.225009,-0.137750,-0.150475,0.190466,-0.142317,-0.153234,-0.211302,0.104903,-0.163166,-0.185760,-0.188961,0.205577,0.162633,-0.236461,239
179,-0.051684,0.045323,0.026551,0.084307,-0.011685,0.098770,0.035624,-0.088679,0.047632,-0.103323,-0.065182,-0.017486,0.100778,0.112023,-0.020218,0.067689,-0.027486,0.059353,0.026200,-0.031356,-0.046602,0.031782,-0.019731,-0.064642,-0.021312,-0.083652,0.046627,0.072482,0.050249,0.062772,0.060120,-0.021497,-0.039408,0.059343,0.102907,-0.086575,-0.080573,0.100881,0.080254,-0.106813,0.039124,0.030498,0.054966,-0.048235,0.027666,0.095903,0.093906,-0.065509,-0.031822,0.006457,179
302,-0.049619,0.045235,0.058379,0.051550,-0.057101,0.077846,0.023243,-0.081356,0.045606,-0.067361,0.001763,0.002096,0.086398,0.013075,-0.020535,0.039203,-0.084504,0.044779,0.003284,-0.045027,-0.067333,-0.009035,-0.067773,-0.062386,0.011005,-0.062871,0.018533,0.057700,0.066586,0.050256,0.032216,-0.047002,-0.011253,0.051189,0.024332,-0.029509,-0.053771,0.085987,-0.004906,-0.021085,0.022138,-0.001723,0.032484,-0.032023,0.030106,0.060863,-0.002142,-0.036177,-0.008199,0.042043,302
0,-0.073702,0.032564,-0.013251,0.040774,-0.025088,0.041930,0.050511,-0.027606,0.037461,-0.006210,0.004907,-0.027966,0.029291,0.021868,0.007239,0.015117,-0.058583,0.033997,0.010608,-0.063622,0.005722,0.009801,0.025829,-0.019528,-0.065273,-0.048533,0.057404,0.038707,0.018894,0.060713,0.023861,-0.103749,-0.001826,0.086265,0.010682,-0.041484,-0.023250,0.045459,-0.001510,0.006322,-0.015940,0.048930,0.040362,-0.013443,0.050593,0.037688,0.015798,-0.040677,0.021905,0.006668,0


In [31]:
#merge back in player_names and ids
shooter_tsne = pd.DataFrame(shooter_tsne)
shooter_tsne['pid'] = shooter_embeddings['pid']
shooter_tsne['pid'] = shooter_tsne['pid'].astype(int)

In [84]:
shooter_embeddings.shape

(875, 51)

In [32]:
#merge in names
shooter_tsne_named = pd.merge(shooter_tsne, player_pid_mapping, on='pid')

In [33]:
shooter_embeddings['pid'] = shooter_embeddings.pid.astype(int)
shooter_embeddings = pd.merge(shooter_embeddings, player_pid_mapping, on='pid')

In [34]:
shooter_embeddings.head()

,e1,e2,e3,e4,e5,e6,e7,e8,e9,e10,e11,e12,e13,e14,e15,e16,e17,e18,e19,e20,e21,e22,e23,e24,e25,e26,e27,e28,e29,e30,e31,e32,e33,e34,e35,e36,e37,e38,e39,e40,e41,e42,e43,e44,e45,e46,e47,e48,e49,e50,pid,name
0,0.145861,0.060942,-0.162317,0.116521,0.102375,-0.122120,-0.082303,0.034962,0.086558,0.091245,-0.120297,-0.050057,-0.183504,-0.117833,0.034199,-0.056183,-0.098333,-0.024634,0.049995,-0.046039,0.040207,-0.093161,0.094091,0.061611,0.094433,-0.086497,0.039598,0.105978,0.048808,0.049360,0.097101,0.082115,0.077424,0.058181,-0.090922,0.101726,0.044492,0.088789,-0.081331,0.098527,0.129745,-0.108056,-0.097394,0.068444,0.054562,-0.167739,0.107999,-0.062441,0.124299,-0.088717,96,David.Pastrnak
1,0.027162,-0.070893,0.027721,-0.050814,-0.041862,0.059687,0.072320,-0.088384,-0.044163,0.030299,0.056952,0.038949,-0.020898,-0.004208,-0.095019,0.050847,0.082287,0.042993,-0.008876,0.026541,-0.098129,0.063506,-0.070664,-0.063470,-0.063600,0.086194,-0.033844,0.080197,-0.111511,0.017512,-0.080008,0.032316,-0.037722,-0.038124,0.068845,-0.036078,-0.051056,-0.108139,0.025899,-0.096434,-0.027515,0.101593,0.067431,-0.038790,-0.043204,-0.045183,-0.027483,0.033795,-0.016186,-0.010879,179,Nathan.MacKinnon
2,0.268365,0.229462,-0.212464,0.253238,0.279540,-0.258065,-0.279415,0.258875,0.240509,0.242060,-0.226719,-0.257509,-0.257206,-0.234686,0.222459,-0.253505,-0.280366,-0.296878,0.246198,-0.239777,0.244030,-0.261487,0.280880,0.271278,0.247669,-0.307330,0.250398,0.308837,0.305877,0.224752,0.245398,0.253656,0.250657,0.267921,-0.252904,0.281080,0.281106,0.310864,-0.289701,0.268175,0.224581,-0.288116,-0.237987,0.247059,0.241400,-0.247237,0.221769,-0.262184,0.267216,-0.166358,239,Connor.McDavid
3,-0.088892,0.049287,-0.008095,0.050189,0.038796,-0.001803,0.030166,-0.004872,0.031296,0.028977,-0.007679,-0.056806,0.015333,0.030285,-0.039920,-0.022837,-0.042695,0.022855,-0.024501,0.045643,0.023810,-0.050221,0.016176,-0.030571,0.053614,-0.022395,-0.001223,0.045490,0.047564,0.011430,-0.025801,-0.011130,-0.030625,0.036144,0.036815,0.048444,0.015780,0.007020,-0.004680,0.033442,0.028940,-0.018325,0.054501,-0.001842,-0.045956,0.016427,-0.019842,-0.017858,0.002855,-0.001432,337,Matthew.Tkachuk
4,-0.042943,-0.030247,-0.013283,0.037826,-0.015664,0.019334,0.027897,0.025692,-0.014122,0.058668,-0.016180,-0.032696,-0.062556,-0.050210,-0.015610,-0.006681,0.003106,0.137039,0.006128,-0.020474,0.014034,0.043212,0.049592,0.022960,-0.030112,-0.043782,0.010503,0.043819,-0.006916,0.056317,0.002551,-0.035403,-0.004829,-0.028073,-0.006933,-0.022408,0.048272,0.006185,-0.050122,-0.018341,0.030431,0.022734,-0.007365,-0.018434,0.062929,0.016293,0.058953,0.029134,-0.011722,-0.083194,302,Jack.Hughes


In [35]:
#write out to csv
shooter_tsne_named.to_csv('shooter_embed_3d.csv')
shooter_embeddings.to_csv('shooter_embed_full.csv')
